In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# StratifiedKFold cross validation to make sure the same proportion of both classes maintained during each sampling process
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.feature_selection import SelectKBest
from yellowbrick.target import FeatureCorrelation
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.figure(figsize = (20, 18))
import xgboost as xgb
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
# hyperopt is hyperparameter optimization by defining an objective function and declaring a search space
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-dec-2021/train.csv")
train_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.columns

In [ ]:
# Drops ID column as it is not required
train_df.drop(["Id"], axis=1, inplace=True)

In [ ]:
# Check for missing values
sum(train_df.isna().sum())

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        c_min = df[col].min()
        
        if col_type != object:
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
# compress the data
train_df = reduce_mem_usage(train_df)

In [ ]:
# Checks distribution of categorical target variable
train_df.groupby(['Cover_Type']).size()

In [ ]:
train_df.drop(train_df[train_df['Cover_Type'] == 5].index, inplace = True) # this has one observation

In [ ]:
# Checks distribution of categorical target variable
train_df.groupby(['Cover_Type']).size()

In [ ]:
# Hillshade_mean
# Mean hillshade of the hillshade at 9AM, Noon, and 3PM (0-255)
train_df['hillshade_mean'] = (train_df['Hillshade_9am']+train_df['Hillshade_Noon']+train_df['Hillshade_3pm'])
train_df.head()

In [ ]:
# Euclidean_Distance_To_Hydrology
# Square root of the sum of the squared horizontal & vertical distances to water
train_df['euclidean_distance_to_hydrology'] = np.sqrt(train_df['Horizontal_Distance_To_Hydrology']**2+train_df['Vertical_Distance_To_Hydrology']**2)
train_df['euclidean_distance_to_hydrology'] = round(train_df['euclidean_distance_to_hydrology'])
train_df['euclidean_distance_to_hydrology'] = np.where(pd.notnull(train_df['euclidean_distance_to_hydrology']), train_df['euclidean_distance_to_hydrology'], 0)
train_df.head()

In [ ]:
train_df['Elevation'].plot()

In [ ]:
#Create log Elevation 
train_df['log_elevation']=np.log10(train_df['Elevation'])

In [ ]:
train_df['log_elevation'].plot()

In [ ]:
train_df['Aspect'] = np.where(train_df['Aspect'] == 360, 0,train_df['Aspect'])

In [ ]:
train_df['elev_hydro_vert_sum'] = train_df['Elevation']+train_df["Vertical_Distance_To_Hydrology"]

In [ ]:
train_df['elev_hydro_horz_sum'] = train_df['Elevation']+train_df["Horizontal_Distance_To_Hydrology"]

In [ ]:
train_df['hyro_lower'] = np.where(train_df["Vertical_Distance_To_Hydrology"]<0, 1,0)

In [ ]:
# features and the target
y = train_df.Cover_Type
X = train_df[['log_elevation','Aspect',"Vertical_Distance_To_Hydrology","Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points","Wilderness_Area1",
              "Wilderness_Area4","Soil_Type2","Soil_Type3","Soil_Type6","Soil_Type10","Soil_Type11","Soil_Type12","Soil_Type13","Soil_Type17",
             "Soil_Type22","Soil_Type23","Soil_Type29","Soil_Type30","Soil_Type38","Soil_Type39","Soil_Type40",'hillshade_mean','euclidean_distance_to_hydrology',
            'elev_hydro_vert_sum','elev_hydro_horz_sum','hyro_lower']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
X[['log_elevation']] = scaler.fit_transform(X[['log_elevation']])
X[['Vertical_Distance_To_Hydrology']] = scaler.fit_transform(X[['Vertical_Distance_To_Hydrology']])
X[['Horizontal_Distance_To_Roadways']] = scaler.fit_transform(X[['Horizontal_Distance_To_Roadways']])
X[['Horizontal_Distance_To_Fire_Points']]= scaler.fit_transform(X[['Horizontal_Distance_To_Fire_Points']])
X[['hillshade_mean']] = scaler.fit_transform(X[['hillshade_mean']])
X[['euclidean_distance_to_hydrology']] = scaler.fit_transform(X[['euclidean_distance_to_hydrology']])
X[['elev_hydro_vert_sum']] = scaler.fit_transform(X[['elev_hydro_vert_sum']])

In [ ]:
X[['elev_hydro_horz_sum']] = scaler.fit_transform(X[['elev_hydro_horz_sum']])

In [ ]:
del train_df

In [ ]:
# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state= 40)

In [ ]:
# Performs cross validation on XGB Classifier

model = XGBClassifier(tree_method='gpu_hist')
model_score = cross_val_score(model, X, y, scoring='accuracy', cv=skf.split(X, y), n_jobs=-1, verbose=10)

In [ ]:
print(model_score.mean())

In [ ]:
del model_score, model

In [ ]:
parameter_space = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (2,10),
    'subsample': (0.4, 1.0),
    'colsample_bytree' :(0.4, 1.0),
    'gamma': (0, 5)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample_bytree,
                        gamma):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)

    clf = XGBClassifier(
        tree_method='gpu_hist',
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma)
    return np.mean(cross_val_score(clf, X, y, cv=5, scoring='accuracy'))

optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=parameter_space,
    random_state=100,
)

In [ ]:
optimizer.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

In [ ]:
optimizer.res

In [ ]:
params_gbm = optimizer.max['params']
params_gbm['max_depth'] = round(params_gbm['max_depth'])
params_gbm['n_estimators'] = round(params_gbm['n_estimators'])
params_gbm

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
params = {'colsample_bytree': 0.8547061059383534,
 'gamma': 4.353355108658212,
 'learning_rate': 0.2502986173540949,
 'max_depth': 8,
 'n_estimators': 940,
 'subsample': 0.740057344466777}
params["max_depth"] = int(params["max_depth"])
params['objective'] = 'multi:softmax'  # error evaluation for multiclass training
params['num_class']=  6  # the number of classes that exist in this datset
params["tree_method"] = "gpu_hist"
params['eval_metric'] =  'mlogloss'

In [ ]:
xgb = XGBClassifier(**params)
xgb.fit(X_train, y_train,
          early_stopping_rounds=200,
          eval_set=[(X_test,y_test)],
          verbose=True)

In [ ]:
y_pred=xgb.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
# Loads test data set
test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")

# Removes ID column as it is not required for prediction
test.drop(["Id"], axis=1, inplace=True)

In [ ]:
# Loads submission data set that acts just as a template for submission
submission = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")

In [ ]:
# Hillshade_mean
# Mean hillshade of the hillshade at 9AM, Noon, and 3PM (0-255)
test['hillshade_mean'] = (test['Hillshade_9am']+test['Hillshade_Noon']+test['Hillshade_3pm'])

In [ ]:
# Euclidean_Distance_To_Hydrology
# Square root of the sum of the squared horizontal & vertical distances to water
test['euclidean_distance_to_hydrology'] = np.sqrt(test['Horizontal_Distance_To_Hydrology']**2+test['Vertical_Distance_To_Hydrology']**2)
test['euclidean_distance_to_hydrology'] = round(test['euclidean_distance_to_hydrology'])
test['euclidean_distance_to_hydrology'] = np.where(pd.notnull(test['euclidean_distance_to_hydrology']), test['euclidean_distance_to_hydrology'], 0)
test.head()

In [ ]:
#Create log Elevation 
test['log_elevation']=np.log10(test['Elevation'])

In [ ]:
test['Aspect'] = np.where(test['Aspect'] == 360, 0,test['Aspect'])

In [ ]:
test['elev_hydro_vert_sum'] = test['Elevation']+test["Vertical_Distance_To_Hydrology"]

In [ ]:
test['elev_hydro_horz_sum'] = test['Elevation']+test["Horizontal_Distance_To_Hydrology"]

In [ ]:
test['hyro_lower'] = np.where(test["Vertical_Distance_To_Hydrology"]<0, 1,0)

In [ ]:
test=test[['log_elevation','Aspect',"Vertical_Distance_To_Hydrology","Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points","Wilderness_Area1",
              "Wilderness_Area4","Soil_Type2","Soil_Type3","Soil_Type6","Soil_Type10","Soil_Type11","Soil_Type12","Soil_Type13","Soil_Type17",
             "Soil_Type22","Soil_Type23","Soil_Type29","Soil_Type30","Soil_Type38","Soil_Type39","Soil_Type40","hillshade_mean",'euclidean_distance_to_hydrology',
           'elev_hydro_vert_sum','elev_hydro_horz_sum','hyro_lower']]

In [ ]:
# transform data
test[['log_elevation']] = scaler.fit_transform(test[['log_elevation']])
test[['Vertical_Distance_To_Hydrology']] = scaler.fit_transform(test[['Vertical_Distance_To_Hydrology']])
test[['Horizontal_Distance_To_Roadways']] = scaler.fit_transform(test[['Horizontal_Distance_To_Roadways']])
test[['Horizontal_Distance_To_Fire_Points']]= scaler.fit_transform(test[['Horizontal_Distance_To_Fire_Points']])
test[['hillshade_mean']] = scaler.fit_transform(test[['hillshade_mean']])
test[['euclidean_distance_to_hydrology']] = scaler.fit_transform(test[['euclidean_distance_to_hydrology']])
test[['elev_hydro_vert_sum']] = scaler.fit_transform(test[['elev_hydro_vert_sum']])
test[['elev_hydro_horz_sum']] = scaler.fit_transform(test[['elev_hydro_horz_sum']])

In [ ]:
predictions = xgb.predict(test)

In [ ]:
submission["Cover_Type"] = predictions

In [ ]:
# Checks for sumbission file before saving
submission

In [ ]:
# Saves test predictions
submission.to_csv("./submission.csv", index=False) #0.92124

In [ ]:
fold_no = 1
for train_index, test_index in skf.split(X, y):
    print('Fold = ',fold_no)
    y_val = y.iloc[test_index]
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    fold_no +=1

In [ ]:
hyperparameter_space = { 
                        'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
                        'max_depth': hp.quniform("max_depth", 2, 6, 1),
                        'min_child_weight' : hp.quniform('min_child_weight', 1, 8, 1),
                        'reg_alpha' : hp.uniform('reg_alpha', 1e-8, 100),
                        'reg_lambda' : hp.uniform('reg_lambda', 1e-8, 100),
                        'gamma': hp.uniform ('gamma', 0.0, 1.0),
                        'subsample': hp.uniform("subsample", 0.1, 1.0),
                        'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1.0)
                       }

In [ ]:
def optimize_hyppara(hyperparameter_space):
    # Converts parameter value to int as required by XGBoost
    hyperparameter_space["max_depth"] = int(hyperparameter_space["max_depth"])
    hyperparameter_space["objective"] = "multi:softmax"
    hyperparameter_space["eval_metric"] = "mlogloss"
    hyperparameter_space["tree_method"] = "gpu_hist"
    hyperparameter_space['num_class']=  6
    
    xgb = XGBClassifier(**hyperparameter_space)
    xgb.fit(X_train, y_train,
          early_stopping_rounds=200,
          eval_set=[(X_test,y_test)],
          verbose=False)
    
    predictions = xgb.predict(X_test)
    
    acc = accuracy_score(y_val, predictions)
    
    del predictions, xgb, hyperparameter_space
    
    return {"loss": -acc, "status": STATUS_OK}

In [ ]:
# Starts hyperparameters tuning
trials = Trials()
best_model_params = fmin(fn=optimize_hyppara,space=hyperparameter_space, max_evals=50,algo=tpe.suggest,trials=trials)

In [ ]:
best_model_params

In [ ]:
params = {'colsample_bytree': 0.8362402722300848,
 'gamma': 0.2483293871916108,
 'learning_rate': 0.24319728150547615,
 'max_depth': 6.0,
 'min_child_weight': 2.0,
 'reg_alpha': 24.64076992525072,
 'reg_lambda': 43.07082727172075,
 'subsample': 0.526670899958752}

params["max_depth"] = int(params["max_depth"])
params['objective'] = 'multi:softmax'  # error evaluation for multiclass training
params['num_class']=  6  # the number of classes that exist in this datset
params["tree_method"] = "gpu_hist"
params['eval_metric'] =  'mlogloss'
    
xgb = XGBClassifier(**params)
xgb.fit(X_train, y_train,
          early_stopping_rounds=200,
          eval_set=[(X_test,y_test)],
          verbose=False)

In [ ]:
# Adds other important parameters
best_model_params["max_depth"] = int(best_model_params["max_depth"])
best_model_params['objective'] = 'multi:softmax'  # error evaluation for multiclass training
best_model_params['num_class']=  6  # the number of classes that exist in this datset
best_model_params["tree_method"] = "gpu_hist"
best_model_params['eval_metric'] =  'mlogloss'

In [ ]:
predictions = xgb.predict(test)

In [ ]:
submission["Cover_Type"] = predictions

# Checks for sumbission file before saving
submission

In [ ]:
# Saves test predictions
submission.to_csv("./submission.csv", index=False) # 0.92072

In [ ]:
del predictions

In [ ]:
# Gets the model trained over cross validation and predictions 
# against each iteration is stored

test_predictions = []

for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print("fold", fold)
    
    xgb = XGBClassifier(**best_model_params)
    xgb.fit(X_train, y_train,
          early_stopping_rounds=200,
          eval_set=[(X_test,y_test)],
          verbose=False)
    
    predictions = xgb.predict(test)
    
    test_predictions.append(predictions)
    
    del predictions, xgb

In [ ]:
test_predictions

In [ ]:
#Predictions stored against each cross validation iteration finally gets aeveraged
# and target column is set with that averaged predictions
submission["Cover_Type"] = np.mean(np.column_stack(test_predictions), axis=1)
submission["Cover_Type"] = submission["Cover_Type"].astype("int32")
# Checks for sumbission file before saving
submission

In [ ]:
# Saves test predictions
submission.to_csv("./submission.csv", index=False) 